In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

/home/user_3/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


/home/user_3/medistream-recsys/Script


In [2]:
df = pd.read_json('/fastcampus-data/log-data/log_sampling.json')

In [3]:
df.head()

,_metadata,anonymousId,channel,context,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,traits,type,userId,version,properties,category,name,event
0,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",623cef3e-6160-4ca8-ab57-f1046a946cb1,client,"{'campaign': {}, 'ip': '220.124.80.98', 'libra...",{},ajs-next-c1583afa7d4c5bf091c633d8232f9da3,2022-05-13T06:28:06.805Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:28:06.815Z,2022-05-13T06:28:06.805Z,2022-05-13 06:28:06.815,{'customerId': None},identify,3471de99bb6b2a801b0b5946389150999832d5132f2c51...,2,None,None,None,None
1,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",623cef3e-6160-4ca8-ab57-f1046a946cb1,client,"{'ip': '220.124.80.98', 'library': {'name': 'a...",{},ajs-next-9b8d710113206ffe6539e3ecc587567e,2022-05-13T06:30:54.677Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:30:54.367Z,2022-05-13T06:30:54.677Z,2022-05-13 06:30:54.367,{'customerId': None},identify,3471de99bb6b2a801b0b5946389150999832d5132f2c51...,2,None,None,None,None
2,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",623cef3e-6160-4ca8-ab57-f1046a946cb1,client,"{'ip': '220.124.80.98', 'library': {'name': 'a...",{},ajs-next-23c3daabb8984bada04b504404bbf197,2022-05-13T06:31:26.179Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:31:25.869Z,2022-05-13T06:31:26.179Z,2022-05-13 06:31:25.869,None,page,3471de99bb6b2a801b0b5946389150999832d5132f2c51...,2,"{'category': '', 'name': 'ArticlesArticle', 'p...",,ArticlesArticle,None
3,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",b50480bc-1b18-449a-a073-39275cea3d82,client,"{'campaign': {}, 'ip': '110.70.46.194', 'libra...",{},ajs-next-04e4e02a6a6444550572ec5e2612e8e5,2022-05-13T06:33:01.601Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:33:02.015Z,2022-05-13T06:33:01.601Z,2022-05-13 06:33:02.015,None,page,7f84cdf255bd825cbe17ec63c7cb0822ff8e5cc49201dc...,2,"{'path': '/', 'referrer': 'https://medistream....",None,None,None
4,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",497f3995-1387-4c97-85af-183bcefac9c9,client,"{'campaign': {}, 'ip': '104.28.102.64', 'libra...",{},ajs-next-4286d9e866e25b67b8451692a02f297a,2022-05-13T06:40:33.953Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:40:34.317Z,2022-05-13T06:40:33.953Z,2022-05-13 06:40:34.317,{'customerId': None},identify,d8d3d56a1089728ddc8e489d5c9144c71d526464d1c756...,2,None,None,None,None


# 딕셔너리 형태 컬럼 세부 시각화

    |--anonymousId X
    |--channel X
    |--integrations X
    |--messageId X
    |--originalTimestamp O
    |--projectId X
    |--receivedAt X
    |--sentAt X
    |--timestamp X
    |--type O
    |--userId O
    |--version X 
    |--category x (값이 하나 '')
    |--name O
    |--event X
    |-- _metadata
        |-- bundled x
        |-- bundledIds x
    |-- context
        |-- campaign
        |   |-- term o (컨텐츠 제목 같음 이상한컨텐츠 드롭하자)
        |   |-- source X (kakaotalk 1 개만 있음)
        |   |-- content X (힌의사 or 학생 컨텐츠를 나타내는 듯)
        |   |-- name O (알림메세지 유입 나타냄)
        |   |-- id X (한개밖에 없음)
        |   |-- medium O (콘텐츠 카테고리 같음)
        |              
        |-- ip x (큰 의미 없을 듯)
        |-- library 
        |   |-- name x
        |   |-- version x
        |              
        |-- locale O
        |-- page 
        |   |-- path O 
        |   |-- referrer x (name 으로 확인 가능)
        |   |-- search X
        |   |   |-- 44개 하위 컬럼 존재(utm)
        |   |   
        |   |-- title X (메디스트림 번호만 있음 뭘 말하는 지 모르겠음)
        |   |-- url O 
        |              
        |-- userAgent  O (모바일, 컴퓨터 확인 가능)
    |-- traits 
        |-- customerId O
    |-- properties
        |-- category x (값이 없음)
        |-- name x
        |-- path X (properties path가 많고 non 없음)
        |-- properties           
        |   |-- version x
        |   |-- type x
        |   |-- category x 
        |   |-- name x
        |              
        |-- referrer x
        |-- search x
        |-- title x (메디스트림 번호만 있음 뭘 말하는 지 모르겠음)
        |-- type x (None이 기존 type 비해 매우 많음)
        |-- url x
        |-- userId x (None이 기존 type 비해 매우 많음)
        |-- version x



# 최종 컬럼
- 분석 이후 최종 컬럼

    |--originalTimestamp O
    |--type O
    |--userId O
    |--name O
    |-- context
        |-- campaign
        |   |-- term o (컨텐츠 제목 같음 이상한컨텐츠 드롭하자)
        |   |-- name O (알림메세지 유입 나타냄)
        |   |-- medium O (콘텐츠 카테고리 같음)         
        |              
        |-- locale O
        |-- page 
        |   |-- path O 
        |   |-- url O 
        |              
        |-- userAgent  O (모바일, 컴퓨터 확인 가능)
    |-- traits 
        |-- customerId O

# 1. 분석 시작

In [4]:
df = pd.read_json('/fastcampus-data/log-data/log_sampling.json')

# originalTimestamp 분석
- 시간은 중요 지표

In [6]:
df.originalTimestamp = pd.to_datetime(df.originalTimestamp)

In [14]:
df.originalTimestamp.head(2)

0    2022-05-13T06:28:06.805Z
1    2022-05-13T06:30:54.677Z
Name: originalTimestamp, dtype: object

# projectId 분석

- 값이 하나밖에 없음 지우자

In [214]:
df.projectId.value_counts()

pAzAoFnKNpLfHxRmZ2oDF4    1019669
Name: projectId, dtype: int64

# type 분석

- page, identify 분석 필요해 보임

In [147]:
df.type.value_counts()

page        554604
identify    464899
track          166
Name: type, dtype: int64

# _metadata

- bundled, bundledIds, unbundled
- 값이 하나 밖에 없어서 전처리

In [132]:
dict_to_set('_metadata')

100%|██████████| 1019669/1019669 [00:00<00:00, 1285646.41it/s]


{'bundled', 'bundledIds', 'unbundled'}

In [228]:
_metadata_columns = ['bundled','bundledIds','unbundled']
_metadata_df = dict_to_column(['_metadata'] , df)[['bundled','bundledIds','unbundled']]

In [229]:
for col in _metadata_columns:
    _metadata_df[col] = _metadata_df[col].apply(lambda x: None if x==None else str(x))


In [232]:
# 모두 값이 하나밖에 없음 전처리
_metadata_df.bundled.value_counts(), _metadata_df.bundledIds.value_counts(), _metadata_df.unbundled.value_counts()

(['Segment.io']    1019669
 Name: bundled, dtype: int64,
 []    1019669
 Name: bundledIds, dtype: int64,
 []    1019669
 Name: unbundled, dtype: int64)

# context

- 

In [133]:
dict_to_set('context')

100%|██████████| 1019669/1019669 [00:07<00:00, 129150.70it/s]


{'campaign', 'ip', 'library', 'locale', 'page', 'userAgent'}

# traits
- cutomerid
- 유저 정보를 담고 있으므로 중요 정보

In [135]:
dict_to_set('traits')

100%|██████████| 1019669/1019669 [00:00<00:00, 1798999.78it/s]


{'customerId'}

# properites
- 결론적으로 무쓸모 정보들이다.
- none 값이 압도적으로 많다.

In [22]:
'''
category : 일치
name     : 5369번 빼고 일치 기존 컬럼을 쓰자
path : 새로움 컬럼
properties : 전부 none 드롭 (5369번 빼고)
referrer : 새로운 컬럼
search : 새로운 컬럼
title : 새로운 컬럼
type : 전부 none 드롭 (5369번 빼고)
url : 새로운 컬럼
userId : 전부 none 드롭 (5369번 빼고)
version : 전부 none 드롭 (5369번 빼고)
'''

def key_set(columns:str):
    key_set = set()
    for i in tqdm(df[columns]):
        if isinstance(i, dict):
            key_set |= set(i.keys())
    return key_set

key_set('properties')

100%|██████████| 1019669/1019669 [00:00<00:00, 1357902.37it/s]


['name',
 'title',
 'properties',
 'url',
 'userId',
 'version',
 'referrer',
 'category',
 'type',
 'path',
 'search']

# context 컬럼과 properties 컬럼 중복 컬럼 확인

In [7]:
def duplicate_confirm_columns(column:str, key:str):
    duplicate_confirm_series = df[column].apply(lambda x: x.get(key, None) if isinstance(x, dict) else None)
    different_cnt = (duplicate_confirm_series.fillna(0) != df[key].fillna(0)).sum()
    different_row = duplicate_confirm_series[(duplicate_confirm_series.fillna(0) != df[key].fillna(0))]
    return print(f'key name : {key}\n' + 'different_cnt :', different_cnt,'\n'+'different_row :', different_row, '\n')

## properites 컬럼 key 확인하기

In [8]:
# keys = key_set('properties')
# properties 컬럼 분해 & 기존 컬럼과 차이 확인
keys = ['category','name','type','userId','version']
for key in keys:
    duplicate_confirm_columns('properties', key)

key name : category
different_cnt : 0 
different_row : Series([], Name: properties, dtype: object) 

key name : name
different_cnt : 1 
different_row : 5369    CheckoutOrderComplete
Name: properties, dtype: object 

key name : type
different_cnt : 1019668 
different_row : 0          None
1          None
2          None
3          None
4          None
           ... 
1019664    None
1019665    None
1019666    None
1019667    None
1019668    None
Name: properties, Length: 1019668, dtype: object 

key name : userId
different_cnt : 1019668 
different_row : 0          None
1          None
2          None
3          None
4          None
           ... 
1019664    None
1019665    None
1019666    None
1019667    None
1019668    None
Name: properties, Length: 1019668, dtype: object 

key name : version
different_cnt : 1019669 
different_row : 0          None
1          None
2          None
3          None
4          None
           ... 
1019664    None
1019665    None
1019666    None
1019667   

# properties 중복 컬럼 na 비교

- type, userId, version(중복 컬럼)
- dropna 통해 남은 row 확인
- 5369번 빼고 전부 nan
- test 용도인 것으로  추정
- properites 컬럼에 있는 type,userId, version은 전처리하자

## type

In [37]:
property_type = df.properties.apply(lambda x: x.get('type', None) if isinstance(x, dict) else None)
property_type.isna().sum(), property_type.dropna()


(1019668,
 5369    page
 Name: properties, dtype: object)

## userId

In [36]:
property_userId = df.properties.apply(lambda x: x.get('userId', None) if isinstance(x, dict) else None)
property_userId.isna().sum(), property_userId.dropna()

(1019668,
 5369    76c6132d47e75c1c12634d133adc4b9037f1da32477df5...
 Name: properties, dtype: object)

## version

In [38]:
property_version = df.properties.apply(lambda x: x.get('version', None) if isinstance(x, dict) else None)
property_version.isna().sum(), property_userId.dropna()

(1019668,
 5369    76c6132d47e75c1c12634d133adc4b9037f1da32477df5...
 Name: properties, dtype: object)

5369번 빼고 전부 nan

In [233]:
# 5369번 확인
# test용도인 것 추정
pd.DataFrame(df.iloc[5369,:]).T

,_metadata,anonymousId,channel,context,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,traits,type,userId,version,properties,category,name,event,bundledIds,bundled,unbundled,_metadata_bundledIds,_metadata_bundled,_metadata_unbundled
5369,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",3c8c87ee-ef2c-470c-8e11-3efe7f07b54a,client,"{'ip': '121.152.176.140', 'library': {'name': ...",{},ajs-next-6366a0e975b9cf851887fa5a9c00b20e,2022-05-13T13:35:34.522Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T13:35:34.911Z,2022-05-13T13:35:34.522Z,2022-05-13 13:35:34.911,None,page,76c6132d47e75c1c12634d133adc4b9037f1da32477df5...,2,"{'name': 'CheckoutOrderComplete', 'path': '/',...",None,None,None,[],[Segment.io],[],[],[Segment.io],[]


# properties 컬럼의 key 컬럼화

### properties_search None 전처리 후 데이터 확인하기

In [63]:
df['search'] = df['search'].apply(lambda x: None if x == {} else x)
search_series = df['search'].dropna()
search_series

### 궁금증
- search 가 none인 것들은 메인 페이지 혹은 메뉴 중 클릭하고 메인페이지에 해당하는 아이템을 클릭한 것일까??
- 비즈니스 분석 용도로는 어떤 검색을 많이하는 지 알수 있고 추천 시스템에 근거가 될 수도 있으므로 중요한 정보일 듯
- 총 100만건(1019669) 요청 중 86만건(863583)은 검색을 안함

In [102]:
df_seach.isnull().sum()

                   1019669
utm_id             1019666
utm_source          992661
c                  1019581
type               1013815
placeholder        1014605
boardId            1017182
boardTitle         1016771
articleId          1019387
shortlink          1019581
imp_uid            1019630
period             1015220
utm_term            992716
debugCustomerId    1019565
subOrder           1019618
paymentMethod      1019656
copyable           1019668
ids                1019145
referrer           1019623
nameType           1017152
af_dp              1019590
deep_link_value    1019581
utm_medium          992468
rank               1019564
imp_success        1019630
sort               1000711
limit              1008889
view               1017533
totalCount         1019618
keyword            1006580
pid                1019581
merchant_uid       1019630
category           1019660
utm_campaign        992664
napp               1019240
utm_content         993103
order              1019618
t

In [121]:
utm_columns = []
for col in df_seach.columns:
    if 'utm' in col:
        utm_columns.append(col)
        
utm_columns

['utm_id',
 'utm_source',
 'utm_term',
 'utm_medium',
 'utm_campaign',
 'utm_content']

In [123]:
df.category.value_counts()

    464889
Name: category, dtype: int64

## integrations 컬럼 분석

- 다 none 이여서 전처리 하자

In [126]:
# {} 값 전처리
df['integrations'] = df['integrations'].apply(lambda x: None if x =={} else x)

In [128]:
df['integrations'].value_counts()

Series([], Name: integrations, dtype: int64)

# 모든 컬럼 dict 검사

In [29]:
for col in df.columns:
    try:
        key = dict_to_set(col,df)
        print(f'{col}:',key)
    except:
        print(col,'downdown')

 14%|█▎        | 138003/1019669 [00:00<00:00, 1378637.66it/s]

anonymousId: set()


 14%|█▍        | 141969/1019669 [00:00<00:00, 1419552.59it/s]

channel: set()


 14%|█▍        | 141122/1019669 [00:00<00:00, 1411016.13it/s]

...

url: set()


 16%|█▌        | 163658/1019669 [00:00<00:00, 1636441.10it/s]

referrer: set()


100%|██████████| 1019669/1019669 [00:00<00:00, 1860497.45it/s]

title: set()


### 검사 결과 page, library,campaign,search 총 4개 만 dict 존재

# context_page 컬럼
- path는 중요한 지표
- productid를 확인할 수 있는 중요한 지표이다.
- properties와 context_page는 search라는 하위 key를 가짐
- 따라서 둘을 비교

In [21]:
context_properites_df = df[['context','properties']]
context_properites_df = dict_to_column(['context','properties'], context_properites_df)
df_context_page = context_properites_df[['page']]
df_context_page = dict_to_column(['page'],df_context_page)

100%|██████████| 1019669/1019669 [00:01<00:00, 893861.28it/s]


In [239]:
# search
# context_page_search 와 properites search 비교

new_search = df_context_page['search'].apply(lambda x: None if x =={} else x)
origin_search = context_properites_df['search'].apply(lambda x: None if x =={} else x)
df_context_page['search'].apply(lambda x: None if x =={} else x).isna().sum(), context_properites_df['search'].apply(lambda x: None if x =={} else x).isna().sum()


(398518, 863583)

In [247]:
# properites_search의 키들 확인
properites_search_key_set = set()
search_df = context_properites_df['search'].apply(lambda x: None if x =={} else x)
for i in search_df:
    if isinstance(i,dict):
        properites_search_key_set |= set(i.keys())
len(properites_search_key_set)

44

In [248]:
# context_page_search 키들 확인
context_page_search_key_set = set()
search_df_origin = df_context_page['search'].apply(lambda x: None if x =={} else x)
for i in search_df_origin:
    if isinstance(i,dict):
        context_page_search_key_set |= set(i.keys())
len(context_page_search_key_set)        

44

In [252]:
# context_page_search와 properties_search는 key는 동일
context_page_search_key_set-properites_search_key_set, properites_search_key_set-context_page_search_key_set

(set(), set())

In [255]:
# 두개 컬럼의 search unique 값들 확인 및 비교
new_search_str = new_search.apply(lambda x : None if x==None else str(x))
origin_search_str = origin_search.apply(lambda x : None if x==None else str(x))

# ne_search_str 더 많이 값들이 있는 것 확인
len(set(new_search_str[~new_search_str.isna()]) - set(origin_search_str[~origin_search_str.isna()])),\
len(set(origin_search_str[~origin_search_str.isna()]) - set(new_search_str[~new_search_str.isna()]))

(12641, 0)

## referrer 비교
- context_page_referrer과 properties_referrer 비교
- context_page 에 있는 것이 더 적다
- context_page_referrer 쓰자

In [256]:
# referrer
len(df_context_page.path.unique()),len(context_properites_df.path.unique())

(15587, 15556)

In [257]:
df_context_page.referrer.isna().sum(), context_properites_df.referrer.isna().sum()

(0, 465065)

## path 비교
- context_page_path 와 properties_path 비교
- properties path가 none 값이 많음
- context path 쓰자

In [258]:
# path
len(df_context_page.path.unique()),len(context_properites_df.path.unique())

(15587, 15556)

In [259]:
df_context_page.path.isna().sum(),context_properites_df.path.isna().sum()

(0, 465065)

## title
- context_page_title 와 properties_path 비교
- 일단 값 자체가 추천에 의미있는 값들이 아님
- title 컬럼은 전처리 진행하자

In [265]:
# context_page title 값 확인해보기
df_context_page.title.value_counts().head()

메디스트림 | Medistream         933829
(1) 메디스트림 | Medistream      33131
(2) 메디스트림 | Medistream      14681
(3) 메디스트림 | Medistream       6603
(13) 메디스트림 | Medistream      4464
Name: title, dtype: int64

In [264]:
# properties title 값 확인해보기
context_properites_df.title.value_counts().head()

메디스트림 | Medistream         511735
(1) 메디스트림 | Medistream      16552
(2) 메디스트림 | Medistream       7331
(3) 메디스트림 | Medistream       3299
(13) 메디스트림 | Medistream      2233
Name: title, dtype: int64

In [260]:
# title
# none 때문에 하나 더 있음
len(df_context_page.title.unique()),len(context_properites_df.title.unique())

(168, 169)

In [54]:
df1.title.isna().sum(),df.title.isna().sum()

(0, 465065)

## url
- context_page_url 와 properties_url 비교
- properties url 은 none 매우 많음
- context_page url을 스자
- path와 용도가 겹치지만 일단 분석해보자

In [266]:
# url
len(df_context_page.url.unique()),len(context_properites_df.url.unique())

(76716, 60386)

In [267]:
df_context_page.url.isna().sum(),context_properites_df.url.isna().sum()

(0, 929788)

# product ids 확인할 수 있는 컬럼 확인(context)
- path, url, referrer 에서 product ids 확인 가능
- path가 가장 유의미하게 확인 가능하였음
- 개수가 다른 이유는 메디스트림 메인페이지 주소로 찍히는 결측치들이 존재하였음

In [17]:
# 제일 잘팔린 products 3개월 
products = ['5ff2d56f5ae455001b219ae2','60d018b7d26af90681b18980','6226b268f967570023fa9ea3','60236e092c593a001ace9ffa','6153c87c4cf9a5002287c4a6']

In [23]:
# path, url, referrer 에서 product ids 확인 가능
len(df_context_page.path[df_context_page.path.str.contains('60d018b7d26af90681b18980')])

38

In [24]:
# url
len(df_context_page.url[df_context_page.url.str.contains('60d018b7d26af90681b18980')])

20

In [25]:
# referrer
len(df_context_page.referrer[df_context_page.referrer.str.contains('60d018b7d26af90681b18980')])

0

# 결론, context_page 컬럼들과 properties 컬럼 비교
- properites 는 none 값이 너무많음
- context_page 안 컬럼들에 정보가 다들어있고 더 많음
- properties는 쓸만한 정보들은 context에 들어있고 이를 사용하자

In [268]:
display(df_context_page.isna().sum())
display(context_properites_df[['path','url','search','referrer','title']].isna().sum())

url         0
path        0
referrer    0
title       0
search      0
dtype: int64

path        465065
url         929788
search      465065
referrer    465065
title       465065
dtype: int64

# context page 외 나머지 컬럼 분석
- library, campaign, locale, ip, userAgent

In [3]:
# 분석위해 data reload
context_df = df[['context']]
context_all_df =  dict_to_column(['context'] ,context_df)

100%|██████████| 1019669/1019669 [00:01<00:00, 877365.20it/s]
/home/user_3/medistream-recsys/Script/preprocessing.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[column].apply(lambda x: x.get(key, None) if isinstance(x, dict) else None if x==None


# context_library_version
- 의미없음 모두 지우자
- 분석하거나 추천 도움될만한 데이터가 아님
- 전처리 진행

In [6]:
context_library_df = dict_to_column(['library'], context_all_df)

100%|██████████| 1019669/1019669 [00:01<00:00, 1000779.87it/s]


In [281]:
context_library_df['version'].value_counts().head()

next-1.41.0    388226
next-1.37.0    233715
next-1.39.1    167033
next-1.38.0     97618
next-1.43.0     83529
Name: version, dtype: int64

# context_library_name
- 값이 하나밖에 없음
- 전처리 진행

In [283]:
context_library_df['name'].value_counts().head()

analytics.js    1019669
Name: name, dtype: int64

# context_campaign 분석
- campaign 안 컬럼들 분석

In [10]:
context_campaign_key_set = set()
for i in context_all_df.campaign:
    if isinstance(i,dict):
        context_campaign_key_set |= set(i.keys())
context_campaign_key_set

{'content', 'id', 'medium', 'name', 'source', 'term'}

In [4]:
# campaign 컬럼들 
context_campaign_df = dict_to_column(['campaign'], context_all_df)

100%|██████████| 1019669/1019669 [00:00<00:00, 1736249.83it/s]


# context_campaign 분석
- loop 로 value_counts를 통해 의미있는 컬럼들인지 분석해보자
- na 값이 많아서 전처리 해야될 듯

In [12]:
context_campaign_all_df = context_campaign_df[list(context_campaign_key_set)]

In [13]:
for col in context_campaign_all_df.columns:
    print(context_campaign_all_df[col].value_counts())

kakaotalk    21129
Name: source, dtype: int64
01GBW6JC74C6XQ3B0V4SN55WR5    1
Name: id, dtype: int64
최저가로 협상해왔습니다. 한의사를 위한 초음파, 아큐비즈 공동구매 시작합니다.                  236
구매 목록 보기                                                     217
한의사로 살면서 없애야 할 생각                                            217
한의사랑 대기업비교.\r                                                186
절판되었던 인기 도서, 이제 만나보실 수 있습니다. <비만문답> 등 가온해미디어 도서 2종 할인 판매.    159
                                                            ... 
다음 카페 인기글 순위에 올라온 전문직 소득\r                                     1
Medistream 허리 진단, 검진 2편 - 척추관 협착증                              1
장마철 고영희씨와 파전 어떠신지?                                             1
선배님들 국시공부가 그렇게 도움이 안되는 공부인가요?                                  1
SLAP, 어떻게 진단할 수 있을까요?                                          1
Name: term, Length: 1224, dtype: int64
sat_14       3395
mon_14       2662
tue_14       2485
thu_14       2258
wed_14       2086
fri_14       2054
mon_17        808
fri_09        778
f

# context_locale 분석

- 있어야됨 한국 유저 대상으로 만 진행해야 일반화 쉬울 듯

In [130]:
context_all_df.locale.value_counts()

ko-KR    868342
ko       132048
ko-kr     17809
en-US       942
en-CA       490
en-PH        19
en-us        10
en-GB         4
ja-JP         3
es-ES         2
Name: locale, dtype: int64

# context_ip 분석

- 있어야할 할듯 특정 트레픽으로만 많이 발생하면 문제라고 판단해야할듯

In [134]:
context_all_df.ip.value_counts()

61.253.178.119     18677
110.8.170.95       17184
220.87.56.106      13381
211.216.221.207    12096
121.143.67.140     11919
                   ...  
118.235.33.101         1
106.102.128.7          1
118.235.5.245          1
223.38.36.186          1
211.234.180.1          1
Name: ip, Length: 13940, dtype: int64

In [135]:
len(context_all_df.ip.unique())

13940

# context_useragent 분석

- 모바일 pc 분석 가능함
- 디바이스 종류는 추천에 있어서 중요한 요인이라고 생각

In [143]:
df.userAgent.value_counts()

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36                                                                                                 61317
Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Mobile/15E148 Safari/604.1                                                                         54355
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36                                                                                                 37197
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36                                                                                                 27679
Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Medistream/True MediAppVersion/1.2

# 2. 컬럼별 분석은 끝
- 이후 none 값 채우기 위한 전처리 및 최종 데이터프레임 만들기

# 최종 컬럼

    |--originalTimestamp O
    |--type O
    |--userId O
    |--name O
    |-- context
        |-- campaign
        |   |-- term o (컨텐츠 제목 같음 이상한컨텐츠 드롭하자)
        |   |-- name O (알림메세지 유입 나타냄)
        |   |-- medium O (콘텐츠 카테고리 같음)    
        |              
        |-- locale O
        |-- page 
        |   |-- path O 
        |   |-- url O 
        |              
        |-- userAgent  O (모바일, 컴퓨터 확인 가능)
    |-- traits 
        |-- customerId O

In [148]:
df = pd.read_json('/fastcampus-data/log-data/log_sampling.json')
# 최종 컬럼
final_columns = ['originalTimestamp','type','userId','name','context','traits']
# 최종 데이터프레임 만들기
df_final_column = df[final_columns]
final_log = key_to_element(['traits'],df_final_column)
final_log = dict_to_column(['context','page'], final_log)
final_all_dict_log = dict_to_column(['campaign'], final_log)

100%|██████████| 1019669/1019669 [00:00<00:00, 1810645.91it/s]


In [156]:
df_final_all_before_rename = final_all_dict_log[['originalTimestamp','type','userId','name','traits','locale','userAgent','url','path','term','medium','campaign_name']]
df_final_all_after_rename = df_final_all_before_rename.rename(columns={'traits':'customerId'})

# userId 와 customerId 비교 분석

In [272]:
df_user_customer = df[['userId','customerId']].copy()

In [273]:
# 딕셔너리 리스트에 두개이상 값이 담기는경우('') 없애기
df_user_customer.customerId = df_user_customer.customerId.apply(lambda x: None if x == '' else x)

In [274]:
# userId
# 전부 값이 채워져 있음"
 
len(df_user_customer.userId.unique()), df_user_customer.userId.isna().sum()

(1054, 0)

In [275]:
# customerId
# Nan 값이 너무 많음 
len(df_user_customer.customerId.unique()), df_user_customer.customerId.isna().sum()

(862, 556793)

In [276]:
# userId로 customerId 맵핑 가능한지 확인하기 
# nan 값 있는 것 확인함
df_customer_none_na = df_user_customer[df_user_customer.customerId.isna()]
df_user_customer[df_user_customer.userId.isin(df_customer_none_na.userId)].customerId

0                              None
1                              None
2                              None
3                              None
4                              None
                     ...           
1019664    5d6fd2b219efa30eb29141b6
1019665                        None
1019666                        None
1019667                        None
1019668    5d610ff44e77525ec5ca151a
Name: customerId, Length: 1019668, dtype: object

In [247]:
#userid 에 1대1 매칭 되도록 만들기 그리고 확인
userId_dict = { i:[] for i in df.userId}
for row in df.itertuples():
    if isinstance(row.customerId,str):
        if row.customerId not in (list(userId_dict[row.userId])):
            userId_dict[row.userId].append(row.customerId)

In [248]:

# 두개 이상 들어간 customer 확인
for i in userId_dict.values():
    if len(i)>1:
        print(i)

In [263]:
# 1대1 매칭 되면 딕셔너리 있는 cutomerid 넣기
for idx, row in enumerate(df.itertuples()):
    if row.customerId == None:
        if len(userId_dict[row.userId]) == 1:
            df.loc[idx,'customerId'] = userId_dict[row.userId][0]

KeyboardInterrupt: 

In [207]:
# 전체 데이터 수
len(df_final_all_after_rename)

1019669

In [157]:
df_final_all_after_rename.isna().sum()

originalTimestamp         0
type                      0
userId                    0
name                 554780
customerId           556763
locale                    0
userAgent                 0
url                       0
path                      0
term                 998684
medium               998342
campaign_name        998413
dtype: int64

## samplig log 유저와 아이템 수

In [205]:
len(df_final_all_after_rename.customerId.unique()), len(df_final_all_after_rename.path.unique())

(863, 15587)

# name의 productsproduct 만 아이템임
- none name 값은 path를 통해 확인할 수 있음

In [196]:
df_final_all_after_rename[df_final_all_after_rename.path.str.contains('product')].isna().sum()

originalTimestamp       0
type                    0
userId                  0
name                 1769
customerId           5563
locale                  0
userAgent               0
url                     0
path                    0
term                 7057
medium               7057
campaign_name        7057
dtype: int64

In [203]:
a = df_final_all_after_rename[df_final_all_after_rename.path.str.contains('product')]
a[a.name.isna()].path

42782      /products/628c5f8f9d9388002406e579
42892      /products/5ff2d56f5ae455001b219ae2
43047      /products/62ff2f43ca740b0024397ca4
43084      /products/62f34a970b5b9e002b2dd52f
43236      /products/5f51fbd9a0b91001bf48f1ec
                          ...                
1018732    /products/5fa35b6751203163343219bf
1018850    /products/608b97c50ccc56001bc7cb41
1019284    /products/620315dbceda5e0024734524
1019317    /products/6220537ef967570023fa7d66
1019627    /products/62fc9add0b5b9e002b2e0488
Name: path, Length: 1769, dtype: object